<a href="https://colab.research.google.com/github/Sug-ar-N-Spice/Dr.Chats/blob/Patricia/Patricia_Dr_chat_pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers
! pip install sacremoses
! pip install transformers
! pip install datasets
! pip install torch
#!pip install -q gradio

In [1]:
import pandas as pd
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from typing import List, Dict
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sentence_transformers import SentenceTransformer
import pickle
import os
from datetime import datetime
import glob
import gc  #garbage collector


In [2]:

##STOP WORDS IN NLP DONT MEAN ANYTHING LIKE WE THEY THEY JUST COMPLETE THE SENTENCE

## THIS IS CLASS THAT Cleans the data
class MEDDataPreprocessor:
    """
    Preprocessor for general medical data.
    This class handles cleaning, normalization, and preparation of text data
    related to medical topics for use in a medical chatbot.
    """

    def __init__(self):
        """Initialize the preprocessor with necessary NLTK downloads."""
        nltk.download('punkt', quiet=True)
        nltk.download('stopwords', quiet=True)
        self.stop_words = set(stopwords.words('english'))

        # Minimal list of medical terms to preserve
        self.medical_terms = {
            # Terms conflicting with stopwords
            'a', 'am', 'an', 'as', 'at', 'be', 'by', 'in', 'no', 'on', 'or', 'to', 'up',
            # Critical abbreviations to always preserve
            'ct', 'dr', 'er', 'hiv', 'hr', 'icu', 'iv', 'mr', 'ms'
        }

        self.stop_words = self.stop_words - self.medical_terms

    def clean_text(self, text: str) -> str:



        """
        Clean and normalize the input text.

        Args:
            text (str): Raw input text

        Returns:
            str: Cleaned and normalized text
        """

        if pd.isna(text):
            return ""

        # Convert to lowercase
        text = text.lower()

        # Remove special characters but keep medical symbols
        text = re.sub(r'[^a-zA-Z0-9\s+\-/%]', '', text)

        # Remove extra whitespace
        text = ' '.join(text.split())

        return text

    def remove_stopwords(self, text: str) -> str:
        """
        Remove stopwords from the text, keeping medical specific terms.

        Args:
            text (str): Input text

        Returns:
            str: Text with stopwords removed
        """
        words = word_tokenize(text) ### this is resulting in a list of words was converting sentence / paragraph into a list of words

        filtered_words = [word for word in words if word.lower() not in self.stop_words]
        return ' '.join(filtered_words)

    def preprocess_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Preprocess the entire dataframe.

        Args:
            df (pd.DataFrame): Input dataframe

        Returns:
            pd.DataFrame: Preprocessed dataframe
        """
        # change columns names depending which csv file you are using cleaning text and removing stopwords
        df['clean_question'] = df['question'].apply(self.clean_text).apply(self.remove_stopwords)
        df['clean_context'] = df['context'].apply(self.clean_text).apply(self.remove_stopwords)

        # Combine cleaned abstract and results or full_texts depending on which csv you are using
        df['combined_text'] = df['clean_question'] + ' ' + df['clean_context']

        return df

    def prepare_for_model(self, text: str, max_length: int = 512) -> str: ##looks at paragraph, cuts the paragraph if more than 512 This takes the sentence splits to words and has a max length
        """
        Prepare text for model input, truncating if necessary.

        Args:
            text (str): Input text
            max_length (int): Maximum number of words

        Returns:
            str: Prepared text
        """
        words = text.split()
        if len(words) > max_length:
            return ' '.join(words[:max_length])
        return text



In [3]:
def preprocess_dataframe(self, df): ###USUALLY YOU SEE SELF IN A CLASS This allows you to code attributes in a class
    """
    Preprocess the entire dataframe.

    Args:
        df (pd.DataFrame or DatasetDict): Input dataframe with 'question' and 'context' columns

    Returns:
        pd.DataFrame: Preprocessed dataframe
    """

    # Check if df is a DatasetDict object-- When we try to find dataset Dict- that means we havent converted it to pandas and cleaning whole dataset


    if isinstance(df.DatasetDict):
        # Process each split separately and combine into a DataFrame
        all_data = []
        for split in df:
                    # Assuming all splits have the same columns
                    # Process the columns in each split
            df_split = df[split].to_pandas()  # Convert to DataFrame

            df_split['clean_question'] = df_split['question'].apply(self.clean_text).apply(self.remove_stopwords)
            df_split['clean_context'] = df_split['context'].apply(self.clean_text).apply(self.remove_stopwords)
            df_split['combined_text'] = df_split['clean_question'] + ' ' + df_split['clean_context']

            all_data.extend(df_split.to_dict('records')) # Add processed data to the list

        processed_df= pd.DataFrame(all_data) # Create a new DataFrame from the combined data

    else:  # If it's a regular DataFrame, process as before
        processed_df= df.copy()  # Create a copy to avoid modifying the original DataFrame
        processed_df['clean_question'] = processed_df['question'].apply(self.clean_text).apply(self.remove_stopwords)
        processed_df['clean_context'] = processed_df['context'].apply(self.clean_text).apply(self.remove_stopwords)
        processed_df['combined_text'] = processed_df['clean_question'] + ' ' + processed_df['clean_context']

    return processed_df  # Return the processed data

In [4]:
class SlidingWindowSelector:
    def __init__(self, model_name='all-MiniLM-L6-v2', window_size=500, stride=250, batch_size=16):
        self.model = SentenceTransformer(model_name)
        self.contexts = []
        self.embeddings = []
        self.window_size = window_size
        self.stride = stride
        self.batch_size = batch_size

    def fit(self, contexts):
        if not contexts:
            print("Warning: Empty contexts provided to fit")
            return



        print(f"Fitting {len(contexts)} contexts")
        for i in range(0, len(contexts), self.stride):
            window = contexts[i:i+self.window_size]
            self.contexts.extend(window)

            # Process embeddings in batches
            for j in range(0, len(window), self.batch_size):
                batch = window[j:j+self.batch_size]
                batch_embeddings = self.model.encode(batch)
                self.embeddings.extend(batch_embeddings)

        self.embeddings = np.array(self.embeddings)
        print(f"After fit: {len(self.contexts)} contexts, embeddings shape: {self.embeddings.shape}")

    def update(self, new_contexts):
        if not new_contexts:  # Check if new_contexts is empty
            print("Warning: Received empty contexts, skipping update")
            return

        print(f"Updating with {len(new_contexts)} new contexts")
        print(f"Before update: {len(self.contexts)} contexts, embeddings shape: {self.embeddings.shape if isinstance(self.embeddings, np.ndarray) else 'empty'}")

        new_embeddings = []
        for i in range(0, len(new_contexts), self.batch_size):
            batch = new_contexts[i:i+self.batch_size]
            batch_embeddings = self.model.encode(batch)
            new_embeddings.extend(batch_embeddings)

        # Convert new_embeddings to numpy array
        new_embeddings = np.array(new_embeddings)
        print(f"New embeddings shape: {new_embeddings.shape}")

        # Check if we have existing embeddings
        if len(self.embeddings) == 0 or not isinstance(self.embeddings, np.ndarray):
            self.embeddings = new_embeddings
        else:
            # Ensure dimensions match before stacking
            if new_embeddings.size > 0:  # Only stack if we have new embeddings
                try:
                    self.embeddings = np.vstack([self.embeddings, new_embeddings])
                except ValueError as e:
                    print(f"Error in update: Embedding shapes - existing: {self.embeddings.shape}, new: {new_embeddings.shape}")
                    raise

        self.contexts.extend(new_contexts)
        print(f"After update: {len(self.contexts)} contexts, embeddings shape: {self.embeddings.shape}")

    def get_most_relevant_context(self, question, top_n=3):
        question_embedding = self.model.encode([question])

        # Calculate similarities in batches
        all_similarities = []
        for i in range(0, len(self.embeddings), self.batch_size):
            batch_embeddings = self.embeddings[i:i+self.batch_size]
            batch_similarities = np.dot(batch_embeddings, question_embedding.T).squeeze()
            all_similarities.extend(batch_similarities)

        top_indices = np.argsort(all_similarities)[-top_n:][::-1]
        return [self.contexts[i] for i in top_indices]
     # New methods for pickling functionality
    



In [11]:
class DRChatbot:
    def __init__(self, qa_model="microsoft/BioGPT", window_size=500, stride=250, batch_size=16):
        self.model_name = qa_model  # Store model name for loading from pickle
        self.preprocessor = MEDDataPreprocessor()
        self.qa_tokenizer = AutoTokenizer.from_pretrained(qa_model)
        self.qa_model = AutoModelForCausalLM.from_pretrained(qa_model)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.qa_model.to(self.device)
        self.context_selector = SlidingWindowSelector(window_size=window_size, stride=stride, batch_size=batch_size)

    def preprocess_data(self, df):
        processed_df = self.preprocessor.preprocess_dataframe(df)
        return processed_df['combined_text'].tolist()

    def fit_context_selector(self, all_contexts):
        self.context_selector.fit(all_contexts)

    def update_context_selector(self, new_contexts):
        self.context_selector.update(new_contexts)

    def answer_question(self, question):
        relevant_contexts = self.context_selector.get_most_relevant_context(question, top_n=3)
        combined_context = " ".join(relevant_contexts)

        prompt = f"Based on the following medical information:\n{combined_context}\n\nQuestion: {question}\nAnswer:"
        inputs = self.qa_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(self.device)

        input_length = inputs["input_ids"].shape[1]
        max_new_tokens = min(100, 1024 - input_length)

        with torch.no_grad():
            output = self.qa_model.generate(
                inputs["input_ids"],
                max_new_tokens=max_new_tokens,
                num_return_sequences=1,
                do_sample=True,
                temperature=0.7
            )

        return self.qa_tokenizer.decode(output[0], skip_special_tokens=True)

    def save_state(self, save_dir: str, prefix: str = "chunk"):

        """Centralized save method"""

        os.makedirs(save_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save complete state in a single file
        state = {
        'model_name': self.model_name,
        'device': str(self.device),
        'timestamp': timestamp,
        'context_selector': {
            'contexts': self.context_selector.contexts,
            'embeddings': self.context_selector.embeddings,
            'window_size': self.context_selector.window_size,
            'stride': self.context_selector.stride,
            'batch_size': self.context_selector.batch_size
            }
        }
    
        # Save as a single file
        state_path = f"{save_dir}/{prefix}_{timestamp}.pkl"
        with open(state_path, 'wb') as f:
            pickle.dump(state, f)
    
        print(f"Saved complete state to {state_path}")
        return timestamp

    @classmethod
    def load_state(cls, save_dir: str, timestamp: str, prefix: str = "chunk"):
        """Load a chatbot state from saved files"""
        # First try to find the exact file
        files = os.listdir(save_dir)
        matching_files = [f for f in files if timestamp in f and f.startswith(prefix)]
    
        if not matching_files:
            raise FileNotFoundError(f"No checkpoint file found with timestamp {timestamp}")
    
        # Use the actual filename that was found
        actual_file = matching_files[0]
        state_path = f"{save_dir}/{actual_file}"
    
        print(f"Loading state from {state_path}")
    
        with open(state_path, 'rb') as f:
            state = pickle.load(f)
    
        # Create new chatbot instance
        chatbot = cls(qa_model=state['model_name'])
    
        # Restore context selector state
        chatbot.context_selector.contexts = state['context_selector']['contexts']
        chatbot.context_selector.embeddings = state['context_selector']['embeddings']
        chatbot.context_selector.window_size = state['context_selector']['window_size']
        chatbot.context_selector.stride = state['context_selector']['stride']
        chatbot.context_selector.batch_size = state['context_selector']['batch_size']
    
        print(f"After loading: {len(chatbot.context_selector.contexts)} contexts and embeddings shape {chatbot.context_selector.embeddings.shape}")
    
        return chatbot



In [6]:
def get_latest_timestamp(save_dir):
    """Simplified timestamp retrieval"""
    # Look for regular chunk files (excluding error files)
    files = os.listdir(save_dir)
    chunk_files = [f for f in files if f.startswith('chunk_') and not f.startswith('error')]
    
    if not chunk_files:
        return None
    
    # Get the latest file (based on timestamp in filename)
    latest_file = max(chunk_files)
    # Extract timestamp from the filename (everything after the second underscore)
    timestamp = latest_file.split('_', 2)[2].replace('.pkl', '')
    
    print(f"Found latest checkpoint: {latest_file}")  # Debug info
    return timestamp

In [7]:
def process_large_csv(file_path, save_dir='processed_chunks', window_size=500,
                     stride=250, batch_size=16, chunk_size=10000, start_chunk=0):
    
    """
    Process large CSV file with automatic saving and ability to resume.
    
    Args:
        file_path (str): Path to the CSV file
        save_dir (str): Directory to save processed chunks
        window_size (int): Window size for processing
        stride (int): Stride size for processing
        batch_size (int): Batch size for processing
        chunk_size (int): Number of rows to process at once
        start_chunk (int): Chunk number to start/resume from
    """

    os.makedirs(save_dir, exist_ok=True)
    
    # Initialize or load chatbot
    if start_chunk > 0: # If resuming from a later chunk
        latest_timestamp = get_latest_timestamp(save_dir)
        if latest_timestamp:
            print(f"Resuming from checkpoint with timestamp: {latest_timestamp}")
            chatbot = DRChatbot.load_state(save_dir, latest_timestamp)
            print(f"Loaded chatbot with {len(chatbot.context_selector.contexts)} existing contexts")
            print(f"Embeddings shape: {chatbot.context_selector.embeddings.shape}")
        else:
            print("No checkpoint found, starting from beginning")
            chatbot = DRChatbot(window_size=window_size, stride=stride, batch_size=batch_size)

    else: # If starting from the beginning
        chatbot = DRChatbot(window_size=window_size, stride=stride, batch_size=batch_size)
    
    try:
        # Process in chunks with periodic saving
        for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):
            if i < start_chunk:
                continue
                
            contexts = chatbot.preprocess_data(chunk)
            print(f"Preprocessed chunk {i+1}, got {len(contexts)} contexts")
            
            if not contexts:
                print(f"Warning: Chunk {i+1} produced no contexts, skipping")
                continue

            if i == 0:
                print("Fitting initial contexts")
                chatbot.fit_context_selector(contexts)
            else:
                print(f"Updating with chunk {i+1}") # Debug info
                chatbot.update_context_selector(contexts)
            
            # Save every N chunks or on the last chunk
            if (i + 1) % 5 == 0:  # Save every 5 chunks
                chatbot.save_state(save_dir, f"chunk_{i}")
                gc.collect()  # Clean up memory
                
            print(f"Processed chunk {i+1}, total contexts: {len(chatbot.context_selector.contexts)}")
            
    except Exception as e:
        # Save on error
        error_timestamp = chatbot.save_state(save_dir, f"error_chunk_{i}")
        print(f"Error at chunk {i+1}: {str(e)}")
        print(f"Progress saved. To resume, use start_chunk={i}")
        raise
        
    # Final save
    chatbot.save_state(save_dir, "final")
    return chatbot




In [21]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
def main(start_chunk = 0):
    file_path = 'million_sample.csv'
    save_dir = 'Pickle_data_chunks'
    
    try:
        # Start or resume processing
        chatbot = process_large_csv(
            file_path=file_path,
            save_dir=save_dir,
            window_size=500,
            stride=250,
            batch_size=16,
            chunk_size=10000,
            start_chunk=start_chunk  # Change this if resuming from a specific chunk
        )
        
        # Test the chatbot
        test_questions = [
            "What are the symptoms of asthma?",
            "How is diabetes diagnosed?",
            "What are common treatments for hypertension?"
        ]
        
        for question in test_questions:
            answer = chatbot.answer_question(question)
            print(f"\nQ: {question}")
            print(f"A: {answer}")
            
    except KeyboardInterrupt:
        print("\nProcessing interrupted by user. Progress has been saved.")
    except Exception as e:
        print(f"\nAn error occurred: {str(e)}")
        print("Progress has been saved and can be resumed later.")


In [20]:
# Run the main function
main()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\tyzwh\OneDrive\AI\bootcamp\envs\dev\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tyzwh\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Processed chunk 1, total contexts: 19750
Processed chunk 2, total contexts: 29750
Processed chunk 3, total contexts: 39750
Processed chunk 4, total contexts: 49750
Saved complete state to Pickle_data_chunks/chunk_4_20241025_000619.pkl
Saved complete state to Pickle_data_chunks/error_chunk_4_20241025_000620.pkl
Error at chunk 5: name 'gc' is not defined
Progress saved. To resume, use start_chunk=4

An error occurred: name 'gc' is not defined
Progress has been saved and can be resumed later.


In [52]:
main(start_chunk=4)

No checkpoint found, starting from beginning

Processing interrupted by user. Progress has been saved.


In [12]:
# Test the timestamp function
save_dir = 'Pickle_data_chunks'
timestamp = get_latest_timestamp(save_dir)
print(f"Latest timestamp: {timestamp}")

# Then run the main process
main(start_chunk=4)

Found latest checkpoint: chunk_4_20241025_000619.pkl
Latest timestamp: 20241025_000619
Found latest checkpoint: chunk_4_20241025_000619.pkl
Resuming from checkpoint with timestamp: 20241025_000619
Loading state from Pickle_data_chunks/chunk_4_20241025_000619.pkl
After loading: 59750 contexts and embeddings shape (59750, 384)
Loaded chatbot with 59750 existing contexts
Embeddings shape: (59750, 384)
Preprocessed chunk 5, got 10000 contexts
Updating with chunk 5
Updating with 10000 new contexts
Before update: 59750 contexts, embeddings shape: (59750, 384)
New embeddings shape: (10000, 384)
After update: 69750 contexts, embeddings shape: (69750, 384)
Saved complete state to Pickle_data_chunks/chunk_4_20241025_200052.pkl
Processed chunk 5, total contexts: 69750
Preprocessed chunk 6, got 10000 contexts
Updating with chunk 6
Updating with 10000 new contexts
Before update: 69750 contexts, embeddings shape: (69750, 384)
New embeddings shape: (10000, 384)
After update: 79750 contexts, embedding

: 